In [67]:
from collections import Counter
import numpy as np
import pandas as pd
import torch
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from torch import nn, optim
from torch.utils.data import TensorDataset, DataLoader

from task1_Sentiment_Analysis.Config import Config
from task1_Sentiment_Analysis.data_process import pre_process_data, encode_words, encode_data, to_categorical, \
    pad_features

# 数据集
train_data = pd.read_csv(Config.train_path, sep='\t')
test_data = pd.read_csv(Config.test_path, sep='\t')

train_data_pp = pre_process_data(train_data)
test_data_pp = pre_process_data(test_data)


100%|██████████| 66292/66292 [00:06<00:00, 10146.91it/s]


In [83]:
print(len(train_data_pp))
print(len(test_data_pp))

156060
66291


In [8]:
# 构建词典
encode_voc = encode_words(train_data_pp + test_data_pp)
train_data_int = encode_data(train_data_pp, encode_voc)
test_data_int = encode_data(test_data_pp, encode_voc)


100%|██████████| 66292/66292 [00:06<00:00, 9906.87it/s] 


In [10]:
print(len(test_data_int))

66291


In [ ]:
y_target = to_categorical(train_data['Sentiment'], Config.n_classes)  # 将评分转换成one-hot编码

train_reviews_lens = Counter([len(x) for x in train_data_int])

# 从训练集中删除长度为0的评论
non_zero_idx = [ii for ii, review in enumerate(train_data_int) if len(review) != 0]
train_data_int = [train_data_int[ii] for ii in non_zero_idx]
y_target = np.array([y_target[ii] for ii in non_zero_idx])

train_features = pad_features(train_data_int, max(train_reviews_lens))
X_test = pad_features(test_data_int, max(train_reviews_lens))

# 分割数据集为训练集和验证集
X_train, X_val, y_train, y_val = train_test_split(train_features, y_target, test_size=0.2)

In [3]:
print("X_train",X_train.shape)
print("X_val",X_val.shape)
print("X_test",X_test.shape)

X_train (123924, 30)
X_val (30982, 30)
X_test (66291, 30)


In [4]:
# 调整训练集和验证集的大小，使其样本数能够被批量大小（Config.batch_size）整除
train_size = X_train.shape[0] - X_train.shape[0] % Config.batch_size
val_size = X_val.shape[0] - X_val.shape[0] % Config.batch_size
X_train = X_train[:train_size]
X_val = X_val[:val_size]
y_train = y_train[:train_size]
y_val = y_val[:val_size]

In [5]:
ids_test = np.array([t['PhraseId'] for ii, t in test_data.iterrows()])

In [7]:
print(f"X_test shape: {X_test.shape}")
print(f"ids_test shape: {ids_test.shape}")

X_test shape: (66291, 30)
ids_test shape: (66292,)


In [6]:
train_data = TensorDataset(torch.from_numpy(X_train), torch.from_numpy(y_train))
valid_data = TensorDataset(torch.from_numpy(X_val), torch.from_numpy(y_val))
test_data = TensorDataset(torch.from_numpy(X_test), torch.from_numpy(ids_test))

AssertionError: Size mismatch between tensors